In [ ]:
# NEURAL NETWORK STEP BY STEP

# ===== ÉTAPE 1 : Importer les librairies =====
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd

# ===== ÉTAPE 2 : Créer le modèle =====
# MLPClassifier = Multi-Layer Perceptron (réseau de neurones)
# hidden_layer_sizes=(100, 50) = 2 couches cachées avec 100 et 50 neurones
# max_iter=1000 = nombre d'itérations d'entraînement
# random_state=3 = reproductibilité

model = MLPClassifier(
    hidden_layer_sizes=(100, 50),  # Architecture du réseau
    max_iter=1000,                  # Itérations d'entraînement
    random_state=3,
    verbose=1                       # Pour voir la progression
)

# ===== ÉTAPE 3 : Entraîner le modèle =====
# Utilise les données d'entraînement normalisées
print("Entraînement du modèle...")
model.fit(X_train, y_train)
print("Entraînement terminé!\n")

# ===== ÉTAPE 4 : Faire des prédictions =====
y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)

# ===== ÉTAPE 5 : Évaluer les performances =====
print("=" * 50)
print("RÉSULTATS DU NEURAL NETWORK")
print("=" * 50)

train_acc = accuracy_score(y_train, y_pred_train)
test_acc = accuracy_score(y_test, y_pred_test)
test_prec = precision_score(y_test, y_pred_test)
test_recall = recall_score(y_test, y_pred_test)
test_f1 = f1_score(y_test, y_pred_test)

print(f"Train Accuracy : {train_acc:.4f}")
print(f"Test Accuracy  : {test_acc:.4f}")
print(f"Test Precision : {test_prec:.4f}")
print(f"Test Recall    : {test_recall:.4f}")
print(f"Test F1        : {test_f1:.4f}")

# ===== ÉTAPE 6 (OPTIONNEL) : Infos sur le modèle =====
print("\n" + "=" * 50)
print("INFOS SUR LE MODÈLE")
print("=" * 50)
print(f"Nombre de couches : {len(model.hidden_layer_sizes) + 1}")  # +1 pour output
print(f"Nombre de paramètres (poids) : {model.n_layers_}")
print(f"Iterations effectuées : {model.n_iter_}")
print(f"Loss (erreur finale) : {model.loss_:.4f}")